### Data Set: 
https://github.com/mhjabreel/CharCnn_Keras/tree/master/data/ag_news_csv

The AG's news topic classification dataset is constructed by choosing the 4 largest classes from the original corpus. Each class contains 30,000 training samples and 1,900 testing samples. The total number of training samples is 120,000 and testing 7,600. 

The classes are: 

* World
* Sports
* Business
* Science/Technology

#### For more information on how to use Lbl2Vec, visit the [API Guide](https://lbl2vec.readthedocs.io/en/latest/api.html#)

In [1]:
from lbl2vec import Lbl2Vec
import pandas as pd
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import TaggedDocument
from gensim.parsing.preprocessing import strip_tags

C:\Users\Quoc Anh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-06 18:03:24,973	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


### Load data

In [2]:
# load train data
ag_train = pd.read_csv('data/train.csv',sep=',',header=None, names=['class','title','description'])

# load test data
ag_test = pd.read_csv('data/test.csv',sep=',',header=None, names=['class','title','description'])

# load labels with keywords
labels = pd.read_csv('data/labels.csv',sep=';')

# split keywords by separator and save them as array
labels['keywords'] = labels['keywords'].apply(lambda x: x.split(' '))

# convert description keywords to lowercase
labels['keywords'] = labels['keywords'].apply(lambda description_keywords: [keyword.lower() for keyword in description_keywords])

# get number of keywords for each class
labels['number_of_keywords'] = labels['keywords'].apply(lambda row: len(row))

In [3]:
labels

,class_index,class_name,keywords,number_of_keywords
0,1,World,"[election, state, president, police, politics,...",11
1,2,Sports,"[olympic, football, sport, league, baseball, r...",32
2,3,Business,"[company, market, oil, consumers, exchange, bu...",10
3,4,Science/Technology,"[laboratory, computers, science, technology, w...",18


### Tokenize data

In [4]:
# doc: document text string
# returns tokenized document
# strip_tags removes meta tags from the text
# simple preprocess converts a document into a list of lowercase tokens, ignoring tokens that are too short or too long 
# simple preprocess also removes numerical values as well as punktuation characters
def tokenize(doc):
    return simple_preprocess(strip_tags(doc), deacc=True, min_len=2, max_len=15)

In [5]:
# add data set type column
ag_train['data_set_type'] = 'train'
ag_test['data_set_type'] = 'test'

# concat train and test data
ag_full_corpus = pd.concat([ag_train,ag_test]).reset_index(drop=True)

In [6]:
# tokenize and tag documents combined title + description for Lbl2Vec training
ag_full_corpus['tagged_docs'] = ag_full_corpus.apply(lambda row: TaggedDocument(tokenize(row['title'] + '. ' + row['description']), [str(row.name)]), axis=1)

In [7]:
# add doc_key column
ag_full_corpus['doc_key'] = ag_full_corpus.index.astype(str)

In [8]:
# add class_name column
ag_full_corpus = ag_full_corpus.merge(labels, left_on='class', right_on='class_index', how='left').drop(['class', 'keywords'], axis=1)

In [9]:
ag_full_corpus.head()

,title,description,data_set_type,tagged_docs,doc_key,class_index,class_name,number_of_keywords
0,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",train,"([wall, st, bears, claw, back, into, the, blac...",0,3,Business,10
1,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,train,"([carlyle, looks, toward, commercial, aerospac...",1,3,Business,10
2,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,train,"([oil, and, economy, cloud, stocks, outlook, r...",2,3,Business,10
3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,train,"([iraq, halts, oil, exports, from, main, south...",3,3,Business,10
4,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",train,"([oil, prices, soar, to, all, time, record, po...",4,3,Business,10


# Train Lbl2Vec

Train a new model from scratch with the following parameters:
* keywords_list : iterable list of lists with descriptive keywords for each topic.
* tagged_documents : iterable list of [gensim.models.doc2vec.TaggedDocument](https://radimrehurek.com/gensim/models/doc2vec.html#gensim.models.doc2vec.TaggedDocument) elements. Each element consists of one document.
* label_names : iterable list of custom names for each label. Label names and keywords of the same topic must have the same index.
* similarity_threshold : only documents with a higher similarity to the respective description keywords than this treshold are used to calculate the label embedding.
* min_num_docs : minimum number of documents that are used to calculate the label embedding. 
* epochs : number of iterations over the corpus.

In [10]:
# init model with parameters
lbl2vec_model = Lbl2Vec(keywords_list=list(labels['keywords']), tagged_documents=ag_full_corpus['tagged_docs'], label_names=list(labels['class_name']), similarity_threshold=0.30, min_num_docs=100, epochs=10)

In [11]:
# train model
lbl2vec_model.fit()

2023-08-06 18:04:22,106 - Lbl2Vec - INFO - Train document and word embeddings
2023-08-06 18:09:03,734 - Lbl2Vec - INFO - Train label embeddings


# Predict document topics of documents used to train Lbl2Vec

Compute similarity scores of learned document vectors from documents that were used to train the model to each of the learned label vectors. The similarity scores consist of cosine similarities and therefore have a value range of [-1,1].

In [12]:
# predict similarity scores
model_docs_lbl_similarities = lbl2vec_model.predict_model_docs()

2023-08-06 18:09:21,864 - Lbl2Vec - INFO - Get document embeddings from model
2023-08-06 18:09:22,007 - Lbl2Vec - INFO - Calculate document<->label similarities


In [13]:
model_docs_lbl_similarities.head()

,doc_key,most_similar_label,highest_similarity_score,World,Sports,Business,Science/Technology
0,0,Science/Technology,0.471337,0.240753,0.410054,0.421352,0.471337
1,1,Business,0.435966,0.284427,0.320331,0.435966,0.414551
2,2,Business,0.537299,0.214300,0.296246,0.537299,0.460700
3,3,World,0.489334,0.489334,0.129425,0.391338,0.293481
4,4,Business,0.436432,0.379450,0.332522,0.436432,0.409892


# Add label thresholds

Add thresholds for each topic/label. This provides the possibility to select only the documents whose similarity between topic and label is higher than the manually defined thresholds. This way, only confident predictions can be filtered. 

In [14]:
# manually define thresholds for each label
threshold_tpls = [('World', 0.3), ('Sports', 0.35), ('Business', 0.25), ('Science/Technology', 0.2)]

In [15]:
# add the label thresholds to the predicted similarities
threshold_sims = lbl2vec_model.add_lbl_thresholds(lbl_similarities_df=model_docs_lbl_similarities, lbl_thresholds=threshold_tpls)

In [16]:
threshold_sims.head()

,doc_key,most_similar_label,highest_similarity_score,lbl_threshold,World,Sports,Business,Science/Technology
0,0,Science/Technology,0.471337,0.20,0.240753,0.410054,0.421352,0.471337
1,1,Business,0.435966,0.25,0.284427,0.320331,0.435966,0.414551
2,2,Business,0.537299,0.25,0.214300,0.296246,0.537299,0.460700
3,3,World,0.489334,0.30,0.489334,0.129425,0.391338,0.293481
4,4,Business,0.436432,0.25,0.379450,0.332522,0.436432,0.409892


In [18]:
# get only the documents that have a higher similarity to their most similar label 
# than the respective defined threshold
threshold_sims = threshold_sims[threshold_sims['highest_similarity_score'] > threshold_sims['lbl_threshold']]